### Import reviews

In [20]:
%load_ext autoreload
%autoreload 2

import progressbar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    labels = pickle.load(f)
   
entity_names = ['X', 'Y', 'Z']

In [22]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,labels,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [23]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

RelationMention(doc_id=2313: entities=("town"(43:47), "pizza"(75:80), "pizza"(119:124))
i don't live in fountain hills, but was in town at a friends and picked up pizza and wings. wings are not eatable. the pizza did not taste good at all! i won't return and am ticked off at my loss of $20.00. the service however, was great!
[(43, 47), (75, 80), (119, 124)]


In [24]:
import re

def get_words_before(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, quantity)
        return ' '.join(map(str, words[before:index])) 

In [25]:
import re

def get_words_after(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity) +1
        after = index + min(index, quantity)
        return ' '.join(map(str, words[index:after]))

In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

pos_adjectives_list=[]
neu_adjectives_list=[]
neg_adjectives_list=[]

def check_sentiment_adjective(adjective):
    if (sid.polarity_scores(adjective)['compound']) >= 0.1:
        pos_adjectives_list.append(adjective)
        return 'positive'
    elif (sid.polarity_scores(adjective)['compound']) <= -0.1:
        neg_adjectives_list.append(adjective)
        return 'negative'
    else:
        neu_adjectives_list.append(adjective)
        return 'neutral' 

In [27]:
check_sentiment_adjective('normal')

'neutral'

In [28]:
def check_adjectives_before_pos(sentence,entity):
    words = get_words_before(2,sentence,entity)
    if words == None or len(words.split(" ")) == 0:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        for token in doc:
            if token.pos_ == 'ADJ':
                return token.text
                break

In [29]:
def check_adjectives_after_verb(sentence,entity):
    words = get_words_after(3,sentence,entity)
    if words == None or len(words.split(" ")) < 3:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        if doc[0].pos_ == 'VERB' and (doc[1].pos_ == 'ADJ' or doc[2].pos_ == 'ADJ'):
            return doc[1].text

In [30]:
print(get_words_before(2,candidate.text,"chicken"))

None


In [31]:
print(check_adjectives_before_pos(candidate.text,"customer"))

None


In [32]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.8


In [33]:
from babble import Explanation

explanations = []
index = 1


for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
#    print(candidate.text)
    doc = spacy_nlp(candidate.text)
    for token in doc:
        if token.pos_ == 'ADJ':
            adjective = token.text
            explanation = Explanation(
            name='LF_' + str(index),
            label=label,
            condition='the word "' + adjective + '" is in the sentence',
            candidate=candidate,
            )
            explanations.append(explanation)
            index = index + 1
#            print(str(explanation).upper())

| |                         #                      | 1890 Elapsed Time: 0:00:21


In [34]:
for explanation in explanations:
    print(explanation.candidate)

RelationMention(doc_id=1: entities=("love"(0:4), "place"(10:15), "downtown"(16:24))
RelationMention(doc_id=2: entities=("place"(6:11), "staff"(13:18), "service"(90:97))
RelationMention(doc_id=2: entities=("place"(6:11), "staff"(13:18), "service"(90:97))
RelationMention(doc_id=2: entities=("place"(6:11), "staff"(13:18), "service"(90:97))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=3: entities=("owner"(24:29), "problem"(32:39), "wife"(74:78))
RelationMention(doc_id=4: entities=("place"(0:5), "hour"(81:85), "

RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=777: entities=("dessert"(14:21), "place"(28:33), "ice"(106:109))
RelationMention(doc_id=778: entities=("flavor"(7:13), "grocery"(53:60), "store"(61:66))
RelationMention(doc_id=778: entities=("flavor"(7:13), "grocery"(53:60), "store"(61:66))
RelationMention(doc_id=779: entities=("fan"(47:50), "food"(58:62), "beer"(72:76))
RelationMention(doc_id=779: entities=("fan"(47:50), "food"(58:62), "beer"(72:76))
RelationMention(doc_id=779: entities=("fan"(47:50), "food"(58:62), "beer"(72:76))
RelationMention(doc_id=779: entities=("fan"(47:50)

RelationMention(doc_id=1340: entities=("restaurant"(25:35), "hotel"(52:57), "stuff"(114:119))
RelationMention(doc_id=1341: entities=("place"(73:78), "bit"(84:87), "area"(120:124))
RelationMention(doc_id=1341: entities=("place"(73:78), "bit"(84:87), "area"(120:124))
RelationMention(doc_id=1341: entities=("place"(73:78), "bit"(84:87), "area"(120:124))
RelationMention(doc_id=1342: entities=("love"(2:6), "love"(7:11), "order"(34:39))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), "menu"(95:99))
RelationMention(doc_id=1343: entities=("place"(5:10), "bar"(33:36), 

In [35]:
print(explanations)

[Explanation(LF_1: 1, "the word "scottsdale" is in the sentence"), Explanation(LF_2: 1, "the word "awful" is in the sentence"), Explanation(LF_3: 1, "the word "nice" is in the sentence"), Explanation(LF_4: 1, "the word "rude" is in the sentence"), Explanation(LF_5: 1, "the word "rude" is in the sentence"), Explanation(LF_6: 1, "the word "other" is in the sentence"), Explanation(LF_7: 1, "the word "nice" is in the sentence"), Explanation(LF_8: 1, "the word "odd" is in the sentence"), Explanation(LF_9: 1, "the word "great" is in the sentence"), Explanation(LF_10: 1, "the word "rude" is in the sentence"), Explanation(LF_11: 1, "the word "average" is in the sentence"), Explanation(LF_12: 1, "the word "napa" is in the sentence"), Explanation(LF_13: 1, "the word "few" is in the sentence"), Explanation(LF_14: 1, "the word "disappointed" is in the sentence"), Explanation(LF_15: 1, "the word "strawberry" is in the sentence"), Explanation(LF_16: 1, "the word "excited" is in the sentence"), Expla

In [37]:
pos_adjectives_list = list(dict.fromkeys(pos_adjectives_list))
neu_adjectives_list = list(dict.fromkeys(neu_adjectives_list))
neg_adjectives_list = list(dict.fromkeys(neg_adjectives_list))

In [38]:
with open(DATA_FILE3, 'wb') as f:
    pickle.dump(pos_adjectives_list,f)
    
with open(DATA_FILE4, 'wb') as f:
    pickle.dump(neu_adjectives_list,f)

with open(DATA_FILE5, 'wb') as f:
    pickle.dump(neg_adjectives_list,f)

In [39]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

Wrote 9022 explanations to data/my_explanations.tsv
Read 9022 explanations from data/my_explanations.tsv
